In [33]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from sentence_transformers import SentenceTransformer
import langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv


import os

In [28]:
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [3]:
loader=UnstructuredURLLoader(urls=["https://www.icc-cricket.com/news/all-squads-named-so-far-icc-men-s-t20-world-cup-2024","https://en.wikipedia.org/wiki/2024_ICC_Men%27s_T20_World_Cup"])
data=loader.load()

In [4]:
len(data)

2

In [10]:
data

[Document(page_content="All the squads named for the ICC Men's T20 World Cup 2024\n\nNew Zealand were the first team to announce their 15-member squad for the ICC Men's T20 World Cup 2024.\n\nThe ninth edition of the ICC Men's T20 World Cup, which will be a 20-team event, will take place in the USA and West Indies from 1 to 29 June, 2024.\n\nOn Monday, 29 April, New Zealand became the first team to name their squad for the marquee event – they also launched their new playing kit for the tournament on the same day.\n\nPakistan announced their squad on 24 May, meaning all 20 teams have now submitted their squad for the marquee event.\n\nAll squads listed below:\n\nAll the kits revealed so far for Men's T20 World Cup 2024\n\nAfghanistan: Rashid Khan (c), Rahmanullah Gurbaz, Ibrahim Zadran, Azmatullah Omarzai, Najibullah Zadran, Mohammad Ishaq, Mohammad Nabi, Gulbadin Naib, Karim Janat, Nangyal Kharoti, Mujeeb Ur Rahman, Noor Ahmad, Naveen-ul-Haq, Fazalhaq Farooqi, Fareed Ahmad Malik. Rese

In [9]:
text="""
All the squads named for the ICC Men's T20 World Cup 2024\n\nNew Zealand were the first team to announce their 15-member squad for the ICC Men's T20 World Cup 2024.\n\nThe ninth edition of the ICC Men's T20 World Cup, which will be a 20-team event, will take place in the USA and West Indies from 1 to 29 June, 2024.\n\nOn Monday, 29 April, New Zealand became the first team to name their squad for the marquee event – they also launched their new playing kit for the tournament on the same day.\n\nPakistan announced their squad on 24 May, meaning all 20 teams have now submitted their squad for the marquee event.\n\nAll squads listed below:\n\nAll the kits revealed so far for Men's T20 World Cup 2024\n\nAfghanistan: Rashid Khan (c), Rahmanullah Gurbaz, Ibrahim Zadran, Azmatullah Omarzai, Najibullah Zadran, Mohammad Ishaq, Mohammad Nabi, Gulbadin Naib, Karim Janat, Nangyal Kharoti, Mujeeb Ur Rahman, Noor Ahmad, Naveen-ul-Haq, Fazalhaq Farooqi, Fareed Ahmad Malik. Reserves: Sediq Atal, Hazratullah Zazai, Saleem Safi\n\nClick for more details on Afghanistan squad\n\nAustralia: Mitchell Marsh (c), Ashton Agar, Pat Cummins, Tim David, Nathan Ellis, Cameron Green, Josh Hazlewood, Travis Head, Josh Inglis, Glenn Maxwell, Mitchell Starc, Marcus Stoinis, Matthew Wade, David Warner, Adam Zampa. Travelling Reserves: Jake Fraser-McGurk, Matt Short\n\nClick for more details on Australia squad\n\nBangladesh: Najmul Hossain Shanto (c), Taskin Ahmed, Litton Das, Soumya Sarkar, Tanzid Hasan Tamim, Shakib Al Hasan, Tawhid Hridoy, Mahmud Ullah Riyad, Jaker Ali Anik, Tanvir Islam, Shak Mahedi Hasan, Rishad Hossain, Mustafizur Rahman, Shoriful Islam, Tanzim Hasan Sakib. Travelling reserves: Afif Hossain, Hasan Mahmud\n\nClick for more details on Bangladesh squad\n\nCanada: Saad Bin Zafar (c), Aaron Johnson, Ravinderpal Singh, Navneet Dhaliwal, Kaleem Sana, Dilon Heyliger, Jeremy Gordon, Nikhil Dutta, Pargat Singh, Nicholas Kirton, Rayyankhan Pathan, Junaid Siddiqui, Dilpreet Bajwa, Shreyas Movva, Rishiv Joshi. Reserves: Tajinder Singh, Aaditya Varadharajan, Ammar Khalid, Jatinder Matharu, Parveen Kumar\n\nClick for more details on Canada squad\n\nEngland squad: Jos Buttler (c), Moeen Ali, Jofra Archer, Jonathan Bairstow, Harry Brook, Sam Curran, Ben Duckett, Tom Hartley, Will Jacks, Chris Jordan, Liam Livingstone, Adil Rashid, Phil Salt, Reece Topley, Mark Wood\n\nClick for more details on England squad\n\nIndia: Rohit Sharma (c), Hardik Pandya, Yashasvi Jaiswal, Virat Kohli, Suryakumar Yadav, Rishabh Pant, Sanju Samson, Shivam Dube, Ravindra Jadeja, Axar Patel, Kuldeep Yadav, Yuzvendra Chahal, Arshdeep Singh, Jasprit Bumrah, Mohd. Siraj. Reserves: Shubman Gill, Rinku Singh, Khaleel Ahmed, Avesh Khan\n\nClick for more details on India squad\n\nVirat Kohli scored an unbeaten 53-ball 82* to help India secure an opening victory against Pakistan in their ICC Men's T20 World Cup 2022 encounter.\n\nIreland: Paul Stirling (c), Mark Adair, Ross Adair, Andrew Balbirnie, Curtis Campher, Gareth Delany, George Dockrell, Graham Hume, Josh Little, Barry McCarthy, Neil Rock, Harry Tector, Lorcan Tucker, Ben White, Craig Young\n\nClick for more details on Ireland squad\n\nNamibia: Gerhard Erasmus (c), Zane Green, Michael Van Lingen, Dylan Leicher, Ruben Trumpelmann, Jack Brassell, Ben Shikongo, Tangeni Lungameni, Niko Davin, JJ Smit, Jan Frylinck, JP Kotze, David Wiese, Bernard Scholtz, Malan Kruger, PD Blignaut\n\nClick for more details on Namibia squad\n\nNepal: Rohit Paudel (c), Aasif Sheikh, Anil Kumar Sah, Kushal Bhurtel, Kushal Malla, Dipendra Singh Airee, Lalit Rajbanshi, Karan KC, Gulshan Jha, Sompal Kami, Pratis GC, Sundeep Jora, Abinash Bohara, Sagar Dhakal, Kamal Singh Airee\n\nClick for more details on Nepal squad\n\nNetherlands: Scott Edwards (c), Aryan Dutt, Bas de Leede, Kyle Klein, Logan van Beek, Max O’Dowd, Michael Levitt, Paul van Meekeren, Ryan Klein, Saqib Zulfiqar, Sybrand Engelbrecht, Teja Nidamanuru, \u2060\u2060Tim Pringle, Vikram Singh, Viv Kingma, Wesley Barresi. Travelling reserve: Ryan Klein\n\nClick for more details on Netherlands squad\n\nNew Zealand: Kane Williamson (c), Finn Allen, Trent Boult, Michael Bracewell, Mark Chapman, Devon Conway, Lockie Ferguson, Matt Henry, Daryl Mitchell, Jimmy Neesham, Glenn Phillips, Rachin Ravindra, Mitchell Santner, Ish Sodhi, Tim Southee. Travelling Reserve: Ben Sears\n\nClick for more details on New Zealand squad\n\nOman: Aqib Ilyas (c), Zeeshan Maqsood, Kashyap Prajapati, Pratik Athavale, Ayaan Khan, Shoaib Khan, Mohammad Nadeem, Naseem Khushi, Mehran Khan, Bilal Khan, Rafiullah, Kaleemullah, Fayyaz Butt, Shakeel Ahmad, Khalid Kail. Reserves: Jatinder Singh, Samay Shrivastava, Sufyan Mehmood, Jay Odedra\n\nClick for more details on Oman squad\n\nPapua New Guinea: Assadollah Vala (c), Alei Nao, Chad Soper, CJ Amini, Hila Vare, Hiri Hiri, Jack Gardner, John Kariko, Kabua Vagi Morea, Kipling Doriga, Lega Siaka, Norman Vanua, Sema Kamea, Sese Bau, Tony Ura\n\nClick for more details on Papua New Guinea squad\n\nPakistan: Babar Azam (c), Abrar Ahmed, Azam Khan, Fakhar Zaman, Haris Rauf, Iftikhar Ahmed, Imad Wasim, Mohammad Abbas Afridi, Mohammad Amir, Mohammad Rizwan, Naseem Shah, Saim Ayub, Shadab Khan, Shaheen Shah Afridi, Usman Khan\n\nClick for more details on Pakistan squad\n\nScotland: Richie Berrington (c), Matthew Cross, Brad Currie, Chris Greaves, Oli Hairs, Jack Jarvis, Michael Jones, Michael Leask, Brandon McMullen, George Munsey, Safyaan Sharif, Chris Sole, Charlie Tear, Mark Watt, Brad Wheal\n\nClick for more details on Scotland squad\n\nSouth Africa: Aiden Markram (c), Ottniel Baartman, Gerald Coetzee, Quinton de Kock, Bjorn Fortuin, Reeza Hendricks, Marco Jansen, Heinrich Klaasen, Keshav Maharaj, David Miller, Anrich Nortje, Kagiso Rabada, Ryan Rickelton, Tabraiz Shamsi, Tristan Stubbs\n\nClick for more details on South Africa squad\n\nSri Lanka: Wanindu Hasaranga (c), Charith Asalanka, Kusal Mendis, Pathum Nissanka, Kamindu Mendis, Sadeera Samarawickrama, Angelo Mathews, Dasun Shanaka, Dhananjaya De Silva, Maheesh Theekshana, Dunith Wellalage, Dushmantha Chameera, Nuwan Thushara, Matheesha Pathirana, Dilshan Madushanka. Travelling Reserves: Asitha Fernando, Vijayakanth Viyaskanth, Bhanuka Rajapaksa, Janith Liyanage.\n\nClick for more details on Sri Lanka squad\n\nUganda: Brian Masaba (c), Simon Ssesazi, Roger Mukasa, Cosmas Kyewuta, Dinesh Nakrani, Fred Achelam, Kenneth Waiswa, Alpesh Ramjani, Frank Nsubuga, Henry Ssenyondo, Bilal Hassun, Robinson Obuya, Riazat Ali Shah, Juma Miyaji, Ronak Patel. Travelling Reserves: Innocent Mwebaze, Ronald Lutaaya\n\nClick for more details on Uganda squad\n\nUnited States: Monank Patel (c), Aaron Jones, Andries Gous, Corey Anderson, Ali Khan, Harmeet Singh, Jessy Singh, Milind Kumar, Nisarg Patel, Nitish Kumar, Noshtush Kenjige, Saurabh Nethralvakar, Shadley Van Schalkwyk, Steven Taylor, Shayan Jahangir. Reserve Players: Gajanand Singh, Juanoy Drysdale, Yasir Mohammad.\n\nClick for more details on the USA squad\n\nWest Indies: Rovman Powell (c), Alzarri Joseph, Johnson Charles, Roston Chase, Shimron Hetmyer, Jason Holder, Shai Hope, Akeal Hosein, Shamar Joseph, Brandon King, Gudakesh Motie, Nicholas Pooran, Andre Russell, Sherfane Rutherford, Romario Shepherd\n\nClick for more details on the West Indies squad\n\n‘Out of this World’ by Sean Paul and Kes: The official ICC Men’s T20 World Cup 2024 anthem\n\nLimited tickets remain available for all matches in New York, including access to the Premium Club and the exclusive Diamond Club, where fans can rub shoulders with the legends of the game in the best seats in the house. Watch India, Pakistan, Sri Lanka, South Africa, Bangladesh, Canada, Ireland and our hosts USA and be part of history at the Nassau County International Cricket Stadium. Visit tickets.t20worldcup.com to get your tickets.\n\nICC Men's T20 World Cup, 2024\n\nNews\n\nICC Men's T20 World Cup, 2024All the squads named for the ICC Men's T20 World Cup 2024\n\nNewsTwenty equal countries: Inspired Rohit Paudel targets Nepal’s Sri Lanka toppling\n\nICC Men's T20 World Cup, 2024T20 World Cup State of Play: All the scenarios and every side's path to second round\n\nICC Men's T20 World Cup, 2024South Africa beat Bangladesh in record-breaking New York nailbiter
"""

In [13]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

chunks=splitter.split_documents(data)
len(chunks)

71

In [14]:
chunks

[Document(page_content="All the squads named for the ICC Men's T20 World Cup 2024\n\nNew Zealand were the first team to announce their 15-member squad for the ICC Men's T20 World Cup 2024.\n\nThe ninth edition of the ICC Men's T20 World Cup, which will be a 20-team event, will take place in the USA and West Indies from 1 to 29 June, 2024.\n\nOn Monday, 29 April, New Zealand became the first team to name their squad for the marquee event – they also launched their new playing kit for the tournament on the same day.\n\nPakistan announced their squad on 24 May, meaning all 20 teams have now submitted their squad for the marquee event.\n\nAll squads listed below:\n\nAll the kits revealed so far for Men's T20 World Cup 2024", metadata={'source': 'https://www.icc-cricket.com/news/all-squads-named-so-far-icc-men-s-t20-world-cup-2024'}),
 Document(page_content="All squads listed below:\n\nAll the kits revealed so far for Men's T20 World Cup 2024\n\nAfghanistan: Rashid Khan (c), Rahmanullah Gur

In [15]:
#### EMbedding Techinique
embeddings = SentenceTransformerEmbeddings(model_name ='all-MiniLM-L6-v2')

c:\LLMs\NewsResearchTool\news\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\LLMs\NewsResearchTool\news\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
df=FAISS.from_documents(chunks,embeddings)

In [20]:
####Vector Database

query='How many teams are playing in 2024 T20 worldcup'

result=df.similarity_search(query)
result[2].page_content

"The tournament is being contested by a record 20 teams, an expansion from 16 teams at the 2022 tournament; this includes the two hosts, the top eight teams from the 2022 edition, the next two teams in the ICC Men's T20I Team Rankings, and eight teams determined by regional qualifiers. Canada and Uganda qualified for the men's T20 World Cup for the first time, while the United States is participating for the first time by virtue of being co-host.\n\nFormat\n\nThe 20 qualifying teams have been divided into four groups of five teams; the top two teams in each group will advance to the Super 8 round.[1][4] In this stage, the qualifying teams will be split into two groups of four; the top two teams from each group will qualify for the knockout stage, which will consist of two semi-finals and a final.[5]\n\nHost selection"

In [22]:
prompt=ChatPromptTemplate.from_template("""
Answer the Following question based only on  provided context.
think step by step before providing a detailed answer.
<context>
{context}
</context>
Question :{input} """)

In [29]:
llm=ChatGoogleGenerativeAI(model='gemini-pro',temperature=0.2)

In [30]:

### This chain will combine all the Promts documets and pass on to LLM
chain=create_stuff_documents_chain(llm=llm,prompt=prompt)

In [35]:
retriver=df.as_retriever()

In [36]:
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A98AE4FA90>)

In [37]:
####Retrival chain THis takes a User inqueiry ,which is than passed on to retriver to fetch the relavent document those document and input doc will passed to LLM

retrival=create_retrieval_chain(retriver,chain)

In [60]:
retrival.invoke({"input":"How many teams are playing in 2024 T20 worldcup"})

{'input': 'How many teams are playing in 2024 T20 worldcup',
 'context': [Document(page_content="The 2024 ICC Men's T20 World Cup is the ninth edition of the ICC Men's T20 World Cup, a biennial Twenty20 International (T20I) tournament contested by men's national teams and organised by the International Cricket Council (ICC). It is being co-hosted by the West Indies and the United States from 1 to 29 June 2024.[1] It is the first major ICC event tournament to feature matches played in the United States, and in any other country in the Americas outside the West Indies.[2] The West Indies is hosting the T20 World Cup for the second time, after having previously hosted the 2010 edition.[3] England are the defending champions, having defeated Pakistan in the final of the previous edition.", metadata={'source': 'https://en.wikipedia.org/wiki/2024_ICC_Men%27s_T20_World_Cup'}),
  Document(page_content='^ "ICC announces 26 match officials for T20 World Cup". Times of India. 3 May 2024. Retrieve